In [ ]:
!pip install flax jax jaxlib optax
!pip install ftfy regex tqdm
!pip install codecarbon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 925.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.9/494.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 31.0 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
from PIL import Image
from transformers import CLIPProcessor, FlaxCLIPModel, AutoProcessor
import jax
import jax.numpy as jnp

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from codecarbon import EmissionsTracker

tracker = EmissionsTracker(
    project_name="RS-ZSL-UNIVPM",
    measure_power_secs=15,
    output_dir="./emissions-testing",
    output_file="emissions-testing.csv",
    log_level="debug",
    save_to_file=True,
)

tracker.start()

[codecarbon WARNING @ 16:11:06] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 16:11:06] [setup] RAM Tracking...
[codecarbon INFO @ 16:11:06] [setup] GPU Tracking...
[codecarbon INFO @ 16:11:06] No GPU found.
[codecarbon INFO @ 16:11:06] [setup] CPU Tracking...
[codecarbon DEBUG @ 16:11:06] Not using PowerGadget, an exception occurred while instantiating IntelPowerGadget : Platform not supported by Intel Power Gadget
[codecarbon DEBUG @ 16:11:06] Not using the RAPL interface, an exception occurred while instantiating IntelRAPL : Intel RAPL files not found at /sys/class/powercap/intel-rapl on linux
[codecarbon DEBUG @ 16:11:06] Not using PowerMetrics, an exception occurred while instantiating Powermetrics : Platform not supported by Powermetrics
[codecarbon WARNING @ 16:11:06] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 16:11:07] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please co

In [ ]:
# Da modificare per cambiare modello
model_number = "FINAL"

fine_tuned_model_path = "/content/drive/MyDrive/clip_tuned_model/CLIP_" + model_number
dataset_root = "/content/drive/My Drive/dataset-testing/"

print("Using model" + fine_tuned_model_path)

model = FlaxCLIPModel.from_pretrained(fine_tuned_model_path)
#model = FlaxCLIPModel.from_pretrained("flax-community/clip-rsicd-v2")
#model = FlaxCLIPModel.from_pretrained('openai/clip-vit-base-patch32')
processor = AutoProcessor.from_pretrained("flax-community/clip-rsicd-v2")

label_names = ["port", "viaduct", "sparseresidential", "storagetanks", "stadium", "square", "school", "river", "resort", "railwaystation"]

# Create meaningful text prompts for each label
text_prompts = [f"An aerial photo of a {label}" for label in label_names]
text_inputs = processor(text=text_prompts, return_tensors='np', padding=True)

print(label_names)
print(text_prompts)

Using model/content/drive/MyDrive/clip_tuned_model/CLIP_FINAL
['port', 'viaduct', 'sparseresidential', 'storagetanks', 'stadium', 'square', 'school', 'river', 'resort', 'railwaystation']
['An aerial photo of a port', 'An aerial photo of a viaduct', 'An aerial photo of a sparseresidential', 'An aerial photo of a storagetanks', 'An aerial photo of a stadium', 'An aerial photo of a square', 'An aerial photo of a school', 'An aerial photo of a river', 'An aerial photo of a resort', 'An aerial photo of a railwaystation']


In [ ]:
# Function to calculate top k accuracy
def calculate_top_k_accuracy(similarity_scores, ground_truth_label, label_names, k=3):
    # Sort similarity scores and get top k indices
    top_indices = jnp.argsort(-similarity_scores)[0][:k]

    # Convert ground truth label to index
    ground_truth_index = label_names.index(ground_truth_label)

    # Check if ground truth index is in top k indices
    top_k_accuracy = 1.0 if ground_truth_index in top_indices else 0.0

    return top_k_accuracy

In [ ]:
# Initialize variables to accumulate results
total_images = 0
total_top_1_accuracy = 0.0
total_top_3_accuracy = 0.0
total_top_5_accuracy = 0.0

# Load and preprocess images
def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    return processor(images=image, return_tensors='np')['pixel_values']

# Iterate through the dataset and perform inference
for image_name in os.listdir(dataset_root):
    image_path = os.path.join(dataset_root, image_name)
    image = load_image(image_path)

    # Convert image to jax.numpy
    image = jnp.array(image)

    # Compute image and text embeddings
    image_features = model.get_image_features(pixel_values=image)
    text_features = model.get_text_features(**text_inputs)

    # Normalize the embeddings
    image_features = image_features / jnp.linalg.norm(image_features, axis=-1, keepdims=True)
    text_features = text_features / jnp.linalg.norm(text_features, axis=-1, keepdims=True)

    # Compute similarity scores
    similarity_scores = jnp.dot(image_features, text_features.T)

    # Get top 3 labels sorted by similarity
    top_indices = np.argsort(-similarity_scores[0])[:3]
    top_labels = [(label_names[i], similarity_scores[0, i]) for i in top_indices]

    # Extract ground truth label from directory path
    ground_truth_label = image_name.split('_')[0]

    # Calculate top k accuracy for current image
    top_1_accuracy = calculate_top_k_accuracy(similarity_scores, ground_truth_label, label_names, k=1)
    top_3_accuracy = calculate_top_k_accuracy(similarity_scores, ground_truth_label, label_names, k=3)
    top_5_accuracy = calculate_top_k_accuracy(similarity_scores, ground_truth_label, label_names, k=5)

    # Accumulate results
    total_images += 1
    total_top_1_accuracy += top_1_accuracy
    total_top_3_accuracy += top_3_accuracy
    total_top_5_accuracy += top_5_accuracy

    # Compute mean top k accuracy across all images
    mean_top_1_accuracy = total_top_1_accuracy / total_images
    mean_top_3_accuracy = total_top_3_accuracy / total_images
    mean_top_5_accuracy = total_top_5_accuracy / total_images

    # Display results
    print(f'Image: {image_path}')
    print(f'Ground Truth Label: {ground_truth_label}')
    for label, score in top_labels:
        print(f'{label}: {score:.4f}')
    print(f'Top-1 Accuracy: {mean_top_1_accuracy}')
    print(f'Top-3 Accuracy: {mean_top_3_accuracy}')
    print(f'Top-5 Accuracy: {mean_top_5_accuracy}')
    print()

Image: /content/drive/My Drive/dataset-testing/stadium_164.jpg
Ground Truth Label: stadium
stadium: 0.2725
school: 0.2428
railwaystation: 0.2319
Top-1 Accuracy: 1.0
Top-3 Accuracy: 1.0
Top-5 Accuracy: 1.0

Image: /content/drive/My Drive/dataset-testing/square_269.jpg
Ground Truth Label: square
railwaystation: 0.2727
square: 0.2652
sparseresidential: 0.2614
Top-1 Accuracy: 0.5
Top-3 Accuracy: 1.0
Top-5 Accuracy: 1.0

Image: /content/drive/My Drive/dataset-testing/square_276.jpg
Ground Truth Label: square
river: 0.2616
railwaystation: 0.2578
viaduct: 0.2460
Top-1 Accuracy: 0.3333333333333333
Top-3 Accuracy: 0.6666666666666666
Top-5 Accuracy: 1.0

Image: /content/drive/My Drive/dataset-testing/stadium_176.jpg
Ground Truth Label: stadium
stadium: 0.2659
school: 0.2464
railwaystation: 0.2437
Top-1 Accuracy: 0.5
Top-3 Accuracy: 0.75
Top-5 Accuracy: 1.0



[codecarbon INFO @ 16:11:22] Energy consumed for RAM : 0.000020 kWh. RAM Power : 4.753040313720703 W
[codecarbon DEBUG @ 16:11:22] RAM : 4.75 W during 15.01 s [measurement time: 0.0057]
[codecarbon INFO @ 16:11:22] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon DEBUG @ 16:11:22] CPU : 42.50 W during 15.02 s [measurement time: 0.0015]
[codecarbon INFO @ 16:11:22] 0.000197 kWh of electricity used since the beginning.
[codecarbon DEBUG @ 16:11:22] last_duration=15.011221170425415
------------------------


Image: /content/drive/My Drive/dataset-testing/square_299.jpg
Ground Truth Label: square
square: 0.2637
stadium: 0.2584
school: 0.2577
Top-1 Accuracy: 0.6
Top-3 Accuracy: 0.8
Top-5 Accuracy: 1.0

Image: /content/drive/My Drive/dataset-testing/square_27.jpg
Ground Truth Label: square
school: 0.2531
resort: 0.2487
sparseresidential: 0.2421
Top-1 Accuracy: 0.5
Top-3 Accuracy: 0.6666666666666666
Top-5 Accuracy: 1.0

Image: /content/drive/My Drive/dataset-testing/square_70.jpg
Ground Truth Label: square
school: 0.2476
square: 0.2417
resort: 0.2322
Top-1 Accuracy: 0.42857142857142855
Top-3 Accuracy: 0.7142857142857143
Top-5 Accuracy: 1.0

Image: /content/drive/My Drive/dataset-testing/stadium_225.jpg
Ground Truth Label: stadium
stadium: 0.2802
school: 0.2661
railwaystation: 0.2486
Top-1 Accuracy: 0.5
Top-3 Accuracy: 0.75
Top-5 Accuracy: 1.0

Image: /content/drive/My Drive/dataset-testing/stadium_185.jpg
Ground Truth Label: stadium
stadium: 0.2515
school: 0.2474
railwaystation: 0.2357
Top-1 A

[codecarbon INFO @ 16:11:37] Energy consumed for RAM : 0.000040 kWh. RAM Power : 4.753040313720703 W
[codecarbon DEBUG @ 16:11:37] RAM : 4.75 W during 14.99 s [measurement time: 0.0033]
[codecarbon INFO @ 16:11:37] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon DEBUG @ 16:11:38] CPU : 42.50 W during 15.00 s [measurement time: 0.0097]
[codecarbon INFO @ 16:11:38] 0.000394 kWh of electricity used since the beginning.
[codecarbon DEBUG @ 16:11:38] last_duration=14.989928960800171
------------------------


Image: /content/drive/My Drive/dataset-testing/stadium_107.jpg
Ground Truth Label: stadium
railwaystation: 0.2536
stadium: 0.2441
school: 0.2435
Top-1 Accuracy: 0.5384615384615384
Top-3 Accuracy: 0.7692307692307693
Top-5 Accuracy: 0.9230769230769231

Image: /content/drive/My Drive/dataset-testing/square_55.jpg
Ground Truth Label: square
square: 0.2619
railwaystation: 0.2596
sparseresidential: 0.2545
Top-1 Accuracy: 0.5714285714285714
Top-3 Accuracy: 0.7857142857142857
Top-5 Accuracy: 0.9285714285714286

Image: /content/drive/My Drive/dataset-testing/stadium_260.jpg
Ground Truth Label: stadium
stadium: 0.2680
school: 0.2579
railwaystation: 0.2376
Top-1 Accuracy: 0.6
Top-3 Accuracy: 0.8
Top-5 Accuracy: 0.9333333333333333

Image: /content/drive/My Drive/dataset-testing/stadium_217.jpg
Ground Truth Label: stadium
stadium: 0.2660
school: 0.2361
square: 0.2309
Top-1 Accuracy: 0.625
Top-3 Accuracy: 0.8125
Top-5 Accuracy: 0.9375

Image: /content/drive/My Drive/dataset-testing/stadium_109.jpg
G

...

Image: /content/drive/My Drive/dataset-testing/port_310.jpg
Ground Truth Label: port
port: 0.2427
river: 0.2368
resort: 0.2248
Top-1 Accuracy: 0.403250075233223
Top-3 Accuracy: 0.7324706590430334
Top-5 Accuracy: 0.875413782726452

Image: /content/drive/My Drive/dataset-testing/railwaystation_20.jpg
Ground Truth Label: railwaystation
railwaystation: 0.2522
port: 0.2469
river: 0.2349
Top-1 Accuracy: 0.40342960288808666
Top-3 Accuracy: 0.7325511432009627
Top-5 Accuracy: 0.8754512635379061

Image: /content/drive/My Drive/dataset-testing/railwaystation_226.jpg
Ground Truth Label: railwaystation
railwaystation: 0.2816
port: 0.2706
river: 0.2651
Top-1 Accuracy: 0.403609022556391
Top-3 Accuracy: 0.7326315789473684
Top-5 Accuracy: 0.8754887218045113

Image: /content/drive/My Drive/dataset-testing/port_193.jpg
Ground Truth Label: port
port: 0.2622
sparseresidential: 0.2421
river: 0.2420
Top-1 Accuracy: 0.4037883343355382
Top-3 Accuracy: 0.7327119663259171
Top-5 Accuracy: 0.8755261575466026

Imag

[codecarbon INFO @ 18:02:40] Energy consumed for RAM : 0.008819 kWh. RAM Power : 4.753040313720703 W
[codecarbon DEBUG @ 18:02:40] RAM : 4.75 W during 14.97 s [measurement time: 0.0045]
[codecarbon INFO @ 18:02:40] Energy consumed for all CPUs : 0.078928 kWh. Total CPU Power : 42.5 W
[codecarbon DEBUG @ 18:02:40] CPU : 42.50 W during 14.98 s [measurement time: 0.0012]
[codecarbon INFO @ 18:02:40] 0.087747 kWh of electricity used since the beginning.
[codecarbon DEBUG @ 18:02:40] last_duration=14.965481042861938
------------------------


Image: /content/drive/My Drive/dataset-testing/railwaystation_169.jpg
Ground Truth Label: railwaystation
railwaystation: 0.2624
port: 0.2442
river: 0.2433
Top-1 Accuracy: 0.4039039039039039
Top-3 Accuracy: 0.7327327327327328
Top-5 Accuracy: 0.8753753753753754

Image: /content/drive/My Drive/dataset-testing/port_295.jpg
Ground Truth Label: port
river: 0.2567
port: 0.2497
railwaystation: 0.2405
Top-1 Accuracy: 0.40378264785349743
Top-3 Accuracy: 0.7328129690783548
Top-5 Accuracy: 0.8754127889522666

Image: /content/drive/My Drive/dataset-testing/port_373.jpg
Ground Truth Label: port
river: 0.2506
port: 0.2504
railwaystation: 0.2316
Top-1 Accuracy: 0.40366146458583435
Top-3 Accuracy: 0.7328931572629052
Top-5 Accuracy: 0.8754501800720288

Image: /content/drive/My Drive/dataset-testing/railwaystation_113.jpg
Ground Truth Label: railwaystation
river: 0.2948
railwaystation: 0.2864
viaduct: 0.2739
Top-1 Accuracy: 0.40354035403540356
Top-3 Accuracy: 0.7329732973297329
Top-5 Accuracy: 0.87548754

[codecarbon INFO @ 18:02:55] Energy consumed for RAM : 0.008839 kWh. RAM Power : 4.753040313720703 W
[codecarbon DEBUG @ 18:02:55] RAM : 4.75 W during 14.99 s [measurement time: 0.0048]
[codecarbon INFO @ 18:02:55] Energy consumed for all CPUs : 0.079105 kWh. Total CPU Power : 42.5 W
[codecarbon DEBUG @ 18:02:55] CPU : 42.50 W during 15.00 s [measurement time: 0.0018]
[codecarbon INFO @ 18:02:55] 0.087944 kWh of electricity used since the beginning.
[codecarbon DEBUG @ 18:02:55] last_duration=14.98824667930603
------------------------


Image: /content/drive/My Drive/dataset-testing/port_75.jpg
Ground Truth Label: port
railwaystation: 0.2503
river: 0.2471
port: 0.2435
Top-1 Accuracy: 0.4041342121030557
Top-3 Accuracy: 0.7333732774116237
Top-5 Accuracy: 0.8756740563211504

Image: /content/drive/My Drive/dataset-testing/railwaystation_142.jpg
Ground Truth Label: railwaystation
sparseresidential: 0.2724
resort: 0.2674
railwaystation: 0.2371
Top-1 Accuracy: 0.40401317759808325
Top-3 Accuracy: 0.7334531296795448
Top-5 Accuracy: 0.8757112908056305

Image: /content/drive/My Drive/dataset-testing/port_268.jpg
Ground Truth Label: port
port: 0.2643
river: 0.2469
resort: 0.2256
Top-1 Accuracy: 0.4041916167664671
Top-3 Accuracy: 0.7335329341317365
Top-5 Accuracy: 0.875748502994012



In [ ]:
print("Images processed:" + str(total_images))

Images processed:3340


In [ ]:
import pandas as pd
from datetime import datetime
import os

# Define function to save top-k scores to Excel
def save_scores(file_path, mean_top1, mean_top3, mean_top5):
    # Get current date and time
    now = datetime.now()
    date = now.strftime("%Y-%m-%d")
    time = now.strftime("%H:%M:%S")

    # Create new DataFrame with scores and timestamp
    new_data = {'Date': [date], 'Time': [time], 'Top-1 Mean': [mean_top1], 'Top-3 Mean': [mean_top3], 'Top-5 Mean': [mean_top5], 'Model Number': [model_number]}
    df = pd.DataFrame(new_data)

    # Create or load Excel file
    if os.path.exists(file_path):
        existing_df = pd.read_excel(file_path)
        df = pd.concat([existing_df, df], ignore_index=True)
    else:
        df.to_excel(file_path, index=False, sheet_name='Top-K Scores')

    # Save DataFrame back to Excel file
    df.to_excel(file_path, index=False, sheet_name='Top-K Scores')



# File path to your Excel file
excel_file_path = '/content/drive/MyDrive/punteggi.xlsx'

# Save mean top-k scores to Excel
save_scores(excel_file_path, mean_top_1_accuracy, mean_top_3_accuracy, mean_top_5_accuracy)

print(f"Top-k scores saved to {excel_file_path}")

tracker.stop()

[codecarbon INFO @ 18:03:02] Energy consumed for RAM : 0.008848 kWh. RAM Power : 4.753040313720703 W
[codecarbon DEBUG @ 18:03:02] RAM : 4.75 W during 7.00 s [measurement time: 0.0027]
[codecarbon INFO @ 18:03:02] Energy consumed for all CPUs : 0.079188 kWh. Total CPU Power : 42.5 W
[codecarbon DEBUG @ 18:03:02] CPU : 42.50 W during 7.00 s [measurement time: 0.0035]
[codecarbon INFO @ 18:03:02] 0.088036 kWh of electricity used since the beginning.
[codecarbon DEBUG @ 18:03:02] last_duration=6.997231483459473
------------------------
[codecarbon DEBUG @ 18:03:02] EmissionsData(timestamp='2024-06-18T18:03:02', project_name='RS-ZSL-UNIVPM', run_id='9b44a4d1-bbfd-4fcd-9927-7af1d8bdb3e4', duration=6714.72042298317, emissions=0.025134629171645575, emissions_rate=3.7432130585235796e-06, cpu_power=42.5, gpu_power=0.0, ram_power=4.753040313720703, cpu_energy=0.07918776129815311, gpu_energy=0, ram_energy=0.008848316003126805, energy_consumed=0.08803607730127988, country_name='United States', cou

Top-k scores saved to /content/drive/MyDrive/punteggi.xlsx


0.025134629171645575

In [ ]:
tracker.stop()

[codecarbon WARNING @ 18:03:02] Tracker already stopped !
[codecarbon INFO @ 18:03:02] Energy consumed for RAM : 0.008848 kWh. RAM Power : 4.753040313720703 W
[codecarbon DEBUG @ 18:03:02] RAM : 4.75 W during 0.05 s [measurement time: 0.0019]
[codecarbon INFO @ 18:03:02] Energy consumed for all CPUs : 0.079188 kWh. Total CPU Power : 42.5 W
[codecarbon DEBUG @ 18:03:02] CPU : 42.50 W during 0.05 s [measurement time: 0.0018]
[codecarbon INFO @ 18:03:02] 0.088037 kWh of electricity used since the beginning.
[codecarbon DEBUG @ 18:03:02] last_duration=0.04546666145324707
------------------------
[codecarbon DEBUG @ 18:03:02] EmissionsData(timestamp='2024-06-18T18:03:02', project_name='RS-ZSL-UNIVPM', run_id='9b44a4d1-bbfd-4fcd-9927-7af1d8bdb3e4', duration=6714.773418426514, emissions=0.02513481155225325, emissions_rate=3.7432106768158293e-06, cpu_power=42.5, gpu_power=0.0, ram_power=4.753040313720703, cpu_energy=0.07918834006339309, gpu_energy=0, ram_energy=0.008848376040757796, energy_con

0.02513481155225325